In [7]:
import json
import urllib.request
import json
from nltk import word_tokenize
import re
import collections
import pandas as pd
import unidecode

In [8]:

#Movie transcripts
with open("transcripts/" + 'Fotr_parsed_cleaned2.txt' ,encoding='utf-8') as f:
    fellowshipScript = f.read()

with open("transcripts/" + '2t_parsed_and_cleaned.txt' ,encoding='utf-8') as f:
    towersScript = f.read()

with open("transcripts/" + 'RotK_parsed_and_cleaned.txt' ,encoding='utf-8') as f:
    kingScript = f.read()

#Mini transcript for testing
with open("transcripts/" + 'mini_transcript.txt' ,encoding='utf-8') as f:
    mini_script = f.read()



In [9]:
CharDialogue = collections.defaultdict(list)

#Empty character dictionary
def newEmptyCharDict():
    charactersDf = pd.read_csv('characters.csv')

    characters = charactersDf.Name
    for character in characters:
        #unidecode remove accesnts from names, Glóin -> Gloin
        unaccentedChar =  unidecode.unidecode(character)
        #Needed so the Frodo Baggins lines don't end up in the Frodo Gardner list
        if character == "Frodo Gardner":
            CharDialogue['gardner'].append("")
        else:
            CharDialogue[unaccentedChar.lower()].append("")

    #"characters" representing all sentances said by unnamed characters such as "soldier" added
    CharDialogue["hobbits"].append("")
    CharDialogue["ring"].append("")
    CharDialogue["orcs"].append("")
    CharDialogue["men"].append("")



In [10]:
#A mapping function since the names of characters in the wiki does not always match transcripts

def addToCharDict(charName, dialogue):
    charNameSplit = charName.split()
    addSuccessful = False

    for wikiChar in CharDialogue:
        for name in charNameSplit:
            unaccentedName =  unidecode.unidecode(name)
            #'a' from 'a hobbit', also 'and', 'of'
            if len(unaccentedName) < 3 or unaccentedName == "and":
                return
            if unaccentedName in wikiChar:
                CharDialogue[wikiChar].append(dialogue)
                addSuccessful = True
            if unaccentedName == "merry" and "meriadoc brandybuck" == wikiChar:
                CharDialogue["meriadoc brandybuck"].append(dialogue)
                addSuccessful = True
            if unaccentedName == "pippin" and "peregrin took" == wikiChar:
                CharDialogue["peregrin took"].append(dialogue)
                addSuccessful = True
            if unaccentedName == "strider" and "aragorn ii elessar" == wikiChar:
                CharDialogue["aragorn ii elessar"].append(dialogue)
                addSuccessful = True
            if unaccentedName == "galadriel’s" and "galadriel" == wikiChar:
                CharDialogue["galadriel"].append(dialogue)
                addSuccessful = True
            if (unaccentedName == "smeagol") and "gollum" == wikiChar:
                 CharDialogue["gollum"].append(dialogue)
                 addSuccessful = True
            if unaccentedName == "freya" and "freda" == wikiChar:
                CharDialogue["freda"].append(dialogue)
                addSuccessful = True
            if (unaccentedName == "soldier" or unaccentedName == "captain" or unaccentedName == "mother" or unaccentedName == "crowd" or unaccentedName == "gatekeeper" or unaccentedName == "guard" or unaccentedName == "refugees" or unaccentedName == "soldiers") and "men" == wikiChar:
                CharDialogue["men"].append(dialogue)
                addSuccessful = True
    return addSuccessful


In [11]:
#add dialog from a character to a dictionary with characters for keys and all dialogs by character as value
def makeCharacterDialogue(scripts):
    allNotFound = []

    for script in scripts:
        lines = script.splitlines()
        for line in lines:
            splitLine = line.split(":", 1)

            if len(splitLine) > 1:
                dialogue = splitLine[1]
                charName = splitLine[0].lower()
                addChar = addToCharDict(charName, dialogue)
                if addChar == False:
                    allNotFound.append(charName)


In [12]:
scripts = [fellowshipScript, towersScript, kingScript]
newEmptyCharDict()
makeCharacterDialogue(scripts)

#create a json file with the character dialogue
with open("characterDialogue.json", "wt", encoding='utf-8') as outfile:
    json.dump(CharDialogue, outfile, ensure_ascii=False)